In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os


In [2]:
def redo_cs_names(df):
    df.case_study = np.where(df.case_study == "pulp_and_paper", "Pulp & Paper", df.case_study)
    df.case_study = np.where(df.case_study == "iron_and_steel_a", "Steel Plant A", df.case_study)
    df.case_study = np.where(df.case_study == "iron_and_steel_b", "Steel Plant B", df.case_study)    
    df.case_study = np.where(df.case_study == "upw", "Fab25", df.case_study)
    df.case_study = np.where(df.case_study == "ashkelon", "Ashkelon", df.case_study)
    df.case_study = np.where(df.case_study == "carlsbad", "Carlsbad", df.case_study)
    df.case_study = np.where(df.case_study == "santa_barbara", "Santa Barbara", df.case_study)
    df.case_study = np.where(df.case_study == "tampa_bay", "Tampa Bay", df.case_study)
    df.case_study = np.where(df.case_study == "big_spring", "Big Spring", df.case_study)
    df.case_study = np.where(df.case_study == "hrsd", "Hampton Roads", df.case_study)
    df.case_study = np.where(df.case_study == "ocwd", "Orange County", df.case_study)
    df.case_study = np.where(df.case_study == "solaire", "Solaire", df.case_study)
    df.case_study = np.where(df.case_study == "emwd", "EMWD", df.case_study) 
    df.case_study = np.where(df.case_study == "irwin", "Irwin", df.case_study)
    df.case_study = np.where(df.case_study == "kbhdp", "KBHDP", df.case_study)
    df.case_study = np.where(df.case_study == "heap_leaching", "Heap Leaching", df.case_study)
    df.case_study = np.where(df.case_study == "lithium", "Lithium", df.case_study)
    df.case_study = np.where(df.case_study == "uranium", "Uranium", df.case_study)
    df.case_study = np.where(df.case_study == "cherokee", "Cherokee", df.case_study)
    df.case_study = np.where(df.case_study == "gila_river", "Gila River", df.case_study)
    df.case_study = np.where(df.case_study == "beef", "Beef Processing", df.case_study)
    df.case_study = np.where(df.case_study == "monterey_one", "Monterey One", df.case_study)
    df.case_study = np.where(df.case_study == "san_luis", "San Luis TD", df.case_study)
    df.case_study = np.where(df.case_study == "kern_county", "SJC Kern County", df.case_study)
    df.case_study = np.where(df.case_study == "damodar", "Damodar", df.case_study)
    df.case_study = np.where(df.case_study == "produced_water_injection", "SWD Permian Basin", df.case_study)
    
    return df

In [3]:
# source water data
sw = "seawater"
df = pd.read_csv("../results/case_studies/%s/baseline/%s_baseline_results.csv" % (sw, sw))
del df["Unnamed: 0"]

In [4]:
# Constituent table
final_df = pd.DataFrame()
for case_study in df["Case Study"].unique():
    const_in_model = df[((df.Unit == "kg/m3") & (df["Case Study"] == case_study))].Variable.unique()
    
    sw_df = pd.read_csv("../data/case_study_water_sources.csv")
    sw_df = redo_cs_names(sw_df)
    sw_df = sw_df[(sw_df.case_study == case_study)]
    sw_df = sw_df[sw_df.variable.isin(const_in_model)]
    sw_df.variable = sw_df.variable.str.title()
    sw_df.variable = np.where(sw_df.variable == "Tds", "TDS", sw_df.variable)
    sw_df.variable = np.where(sw_df.variable == "Tss", "TSS", sw_df.variable)
    fig = go.Figure(data=[go.Table(header=dict(values=['Constituent', 'Density (mg/L)']),
                     cells=dict(values=[sw_df.variable, round(sw_df.value*1000, 0)]))
                         ])

    xx = sw_df.groupby(["case_study", "water_type", "variable"]).value.sum() * 1000
    xx = xx.reset_index()
    xx.value = xx.value.astype(int)
    final_df = pd.concat([final_df,xx])

final_df.columns = ['Case Study', 'Source Name', 'Constituent', 'Density (mg/L)']
final_df.to_csv("../results/case_studies/SI/Constituent_table_%s.csv" % sw)

In [5]:
# flow rate table
final_df = pd.DataFrame()
for case_study in df["Case Study"].unique():
    sw_df = pd.read_csv("../data/case_study_water_sources.csv")
    sw_df = redo_cs_names(sw_df)
    sw_df = sw_df[(sw_df.case_study == case_study)]
    sw_df.variable = np.where(sw_df.variable == "flow", "Water Flow", sw_df.variable)
    sw_df = sw_df[(sw_df.variable == "Water Flow")]
    
    xx = sw_df.groupby(["case_study", "water_type"]).value.sum()
    xx = xx.reset_index()
    xx.value = xx.value.round(3)
    final_df = pd.concat([final_df,xx])

final_df.columns = ['Case Study', "Source Name", 'Water Flow (m3/s)']
final_df.to_csv("../results/case_studies/SI/WaterFlow_table_%s.csv" % sw)    
    

In [6]:
# basis table
final_df = pd.DataFrame()
for case_study in df["Case Study"].unique():
    
    sw_df = pd.read_csv("../data/case_study_basis.csv")
    sw_df = redo_cs_names(sw_df)
    sw_df = sw_df[(sw_df.case_study == case_study)]
    
    final_df = pd.concat([final_df,sw_df])

del final_df['scenario']; del final_df['reference'];
final_df.columns = ['Case Study', "Source Name", 'Variable']
final_df.to_csv("../results/case_studies/SI/Basis_table_%s.csv" % sw)    

In [70]:
df.head()

,Unit Process Name,Variable,Value,Metric,Unit,Case Study,Scenario,case_name,Treatment Category
0,sw_onshore_intake,Electricity Intensity Unit Inlet,0.102399,Electricity,kwh/m3,Ashkelon,baseline,ashkelonbaselinesw_onshore_intake,Influent Storage and Pumping
1,sw_onshore_intake,Electricity Intensity System Treated,0.232884,Electricity,kwh/m3,Ashkelon,baseline,ashkelonbaselinesw_onshore_intake,Influent Storage and Pumping
2,sw_onshore_intake,Unit Levelized Cost,0.053405,LCOW,$/m3,Ashkelon,baseline,ashkelonbaselinesw_onshore_intake,Influent Storage and Pumping
3,sw_onshore_intake,Total Capital Investment (TCI),35.315344,Cost,$MM,Ashkelon,baseline,ashkelonbaselinesw_onshore_intake,Influent Storage and Pumping
4,sw_onshore_intake,Catalysts and Chemicals,0.000000,Annual Cost,$MM/yr,Ashkelon,baseline,ashkelonbaselinesw_onshore_intake,Influent Storage and Pumping


In [7]:
unit_list = []
d = {}
df1 = df[((df.Unit == "kg") & (df["Case Study"] == case_study))]
df2 = df[((df.Unit == "m3/s") & (df["Case Study"] == case_study))]

# basis table
final_df = pd.DataFrame()
for case_study in df["Case Study"].unique():
    for const in df1.Variable.unique():
        c_list = []
        unit_list = []
        w_list = []
        for unit in df1["Unit Process Name"].unique():
            unit_list.append(unit)
            
            kg_in = df1[((df1["Unit Process Name"] == unit) & (df1.Variable == const) & 
                         (df1.Metric == "Inlet Constituent Total Mass"))].Value.max()
            kg_out = df1[((df1["Unit Process Name"] == unit) & (df1.Variable == const) & 
                          (df1.Metric == "Outlet Constituent Total Mass"))].Value.max()
            c_list.append((1 - (kg_out / kg_in)) * 100)

            w_in = df2[((df2["Unit Process Name"] == unit) & (df2.Variable == "Inlet Water"))].Value.max()
            w_out = df2[((df2["Unit Process Name"] == unit) & (df2.Variable == "Outlet Water"))].Value.max()
            w_list.append(((w_out / w_in)) * 100)            
            
                    
        d[const] = c_list
        d["water recovery"] = w_list
        

    xx = pd.DataFrame(d, index=unit_list).round(1)
    xx["Case Study"] = np.array(case_study)
        
    final_df = pd.concat([final_df,xx])
    
final_df.to_csv("../results/case_studies/SI/ConstWater_removal_table_%s.csv" % sw)    